In [4]:
import pandas as pd
import numpy as np
from numpy import argmax
import matplotlib.pyplot as plt
%matplotlib inline
import librosa
import librosa.display
import IPython.display
import random
import warnings
import os
from PIL import Image
import pathlib
import csv
# sklearn Preprocessing
from sklearn.model_selection import train_test_split
#Keras
import keras
import warnings
warnings.filterwarnings('ignore')
from keras import layers
from keras.layers import Activation, Dense, Dropout, Conv2D, Flatten, MaxPooling2D, GlobalMaxPooling2D, GlobalAveragePooling1D, AveragePooling2D, Input, Add
from keras.models import Sequential
from keras.optimizers import SGD


Collab için drive işlemleri

In [6]:
#drive tanımlaması yapılmak zorunda
from google.colab import drive
drive.mount('/content/drive')



Mounted at /content/drive


In [7]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

 Her ses işlenip spectrogramları png olarak kaydediyor.

In [8]:

classes = 2
#current_path = r"C://Users//Beyza//Desktop//16000_pcm_speeches//Train"
current_path = "drive/My Drive/Colab Notebooks/16000_pcm_speeches/Train"
for i in range (classes):
    pathlib.Path(f'img_data/{i}').mkdir(parents=True, exist_ok=True) #for images
    path = os.path.join(current_path + '/' + str(i)) 
    print("data path:",path)

    datapath = os.listdir(path)  
    print(i,".dosya path->",datapath)
 
    for path_new in datapath:
        path_name =  path +"/" +path_new   
        y, sr = librosa.load(path_name, mono=True, duration=5)
        print(y.shape)
        plt.specgram(y, NFFT=2048, Fs=2, Fc=0, noverlap=128, sides='default', mode='default', scale='dB');
        plt.axis('off');
        plt.savefig((f'img_data/{i}/{path_new[:-3].replace(".", "")}.png'))    #Kaydet
        plt.clf()

data path: drive/My Drive/Colab Notebooks/16000_pcm_speeches/Train/0
0 .dosya path-> ['189.wav', '1429.wav', '21.wav', '1418.wav', '1396.wav', '187.wav', '167.wav', '1409.wav', '150.wav', '1415.wav', '221.wav', '1366.wav', '1385.wav', '1413.wav', '1331.wav', '1428.wav', '1395.wav', '1496.wav', '1441.wav', '1448.wav', '1390.wav', '135.wav', '1478.wav', '133.wav', '1336.wav', '14.wav', '210.wav', '1364.wav', '180.wav', '1398.wav', '1433.wav', '1406.wav', '195.wav', '1485.wav', '1376.wav', '1345.wav', '1468.wav', '1347.wav', '1494.wav', '1447.wav', '207.wav', '1445.wav', '1442.wav', '138.wav', '1435.wav', '205.wav', '1482.wav', '1470.wav', '1419.wav', '1414.wav', '1378.wav', '1452.wav', '1381.wav', '1352.wav', '1365.wav', '1451.wav', '1384.wav', '1399.wav', '151.wav', '1461.wav', '166.wav', '199.wav', '1342.wav', '1476.wav', '1380.wav', '217.wav', '1405.wav', '1463.wav', '1346.wav', '193.wav', '212.wav', '1423.wav', '1325.wav', '1438.wav', '1339.wav', '1437.wav', '1351.wav', '194.wav', '1

<Figure size 432x288 with 0 Axes>

# Yeni Bölüm

In [12]:
!pip install split_folders

In [13]:
#For  dataset
import splitfolders 

splitfolders.ratio('./img_data/', output="./data", seed=1337, ratio=(.8, .2)) # default values
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator( #(X_train , y_train return)
        rescale=1./255,       # all our pixel values are in range (0,1)
        shear_range=0.2,      #to apply some random tranfromations
        zoom_range=0.2,       
        horizontal_flip=True) 

test_datagen = ImageDataGenerator( #(X_test , y_test return)
        rescale=1./255,       # all our pixel values are in range (0,1)
        shear_range=0.2,      #to apply some random tranfromations
        zoom_range=0.2,      
        horizontal_flip=True) 

Copying files: 3000 files [00:00, 7761.16 files/s]


In [15]:
#data generators

#class-mode -> yapılacak sınıflandırmaya göre uygun olanı seçilmelidir. (binary - categorical)     
training_set = train_datagen.flow_from_directory( 
    './data/train',
    target_size=(64, 64),
    batch_size=1,
    class_mode='binary',
    shuffle = False)

test_set = test_datagen.flow_from_directory(
    './data/val',
    target_size=(64, 64),
    batch_size=1,
    class_mode='binary',
    shuffle = False )



Found 2400 images belonging to 2 classes.
Found 600 images belonging to 2 classes.


In [16]:
model = Sequential()
input_shape=(64, 64, 3)                                              #1st hidden layer
model.add(Conv2D(32, (3, 3), strides=(2, 2), input_shape=input_shape))
model.add(AveragePooling2D((2, 2), strides=(2,2)))
model.add(Activation('relu'))                                         #2nd hidden layer
model.add(Conv2D(64, (3, 3), padding="same"))
model.add(AveragePooling2D((2, 2), strides=(2,2)))
model.add(Activation('relu'))                                         #3rd hidden layer
model.add(Conv2D(64, (3, 3), padding="same"))
model.add(AveragePooling2D((2, 2), strides=(2,2)))
model.add(Activation('relu'))                                         #Flatten
model.add(Flatten())
model.add(Dropout(rate=0.5))                                          #Add fully connected layer.
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(rate=0.5))                                          #Output layer
model.add(Dense(10))
model.add(Activation('softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 31, 31, 32)        896       
_________________________________________________________________
average_pooling2d (AveragePo (None, 15, 15, 32)        0         
_________________________________________________________________
activation (Activation)      (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 15, 15, 64)        18496     
_________________________________________________________________
average_pooling2d_1 (Average (None, 7, 7, 64)          0         
_________________________________________________________________
activation_1 (Activation)    (None, 7, 7, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 7, 7, 64)          3

In [17]:
epochs = 200
batch_size = 8
learning_rate = 0.01
decay_rate = learning_rate / epochs
momentum = 0.9
sgd = SGD(lr=learning_rate, momentum=momentum, decay=decay_rate, nesterov=False)
model.compile(optimizer="sgd",  loss ="sparse_categorical_crossentropy", metrics=['accuracy']) #sparse_categorical_crossentropy

In [18]:
model.fit_generator(
        training_set,
        steps_per_epoch= len(training_set), 
        epochs=50,
        validation_data=test_set,
        validation_steps= len(test_set), 
         )

Epoch 1/50
2400/2400 [==============================] - 25s 7ms/step - loss: 0.9817 - accuracy: 0.4623 - val_loss: 0.7041 - val_accuracy: 0.5000
Epoch 2/50
2400/2400 [==============================] - 18s 7ms/step - loss: 0.7381 - accuracy: 0.4809 - val_loss: 0.7216 - val_accuracy: 0.5000
Epoch 3/50
2400/2400 [==============================] - 17s 7ms/step - loss: 0.7292 - accuracy: 0.4909 - val_loss: 0.7352 - val_accuracy: 0.5000
Epoch 4/50
2400/2400 [==============================] - 17s 7ms/step - loss: 0.7227 - accuracy: 0.5052 - val_loss: 0.6955 - val_accuracy: 0.5000
Epoch 5/50
2400/2400 [==============================] - 17s 7ms/step - loss: 0.7176 - accuracy: 0.4907 - val_loss: 0.7238 - val_accuracy: 0.5000
Epoch 6/50
2400/2400 [==============================] - 17s 7ms/step - loss: 0.7101 - accuracy: 0.5032 - val_loss: 0.6681 - val_accuracy: 0.5217
Epoch 7/50
2400/2400 [==============================] - 17s 7ms/step - loss: 0.6324 - accuracy: 0.6538 - val_loss: 0.5467 - val_ac

In [19]:
#Model Evaluation
model.evaluate_generator(generator=test_set, steps=200)

[0.10259240120649338, 0.9800000190734863]

In [33]:
#test veri setini değiştiriyoruz. Ama önce sıfırlanması gerekiyor
test_set.reset()
pred = model.predict_generator(test_set, verbose=1) #steps= len(test_set)

600/600 [==============================] - 3s 5ms/step


In [34]:
predicted_class_indices=np.argmax(pred,axis=1)
#print(predicted_class_indices)

labels = (training_set.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictions = [labels[k] for k in predicted_class_indices]
predictions = predictions[:600]
print(predictions)
filenames=test_set.filenames
#print(filenames)

['0', '0', '0', '0', '0', '0', '1', '0', '0', '0', '0', '0', '0', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1', '0', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1', '0', '0', '0', '0', '0', '0', '0',

In [35]:
print(len(filenames), len(predictions))


600 600


Karşılaştırmalar için tablo oluşturuldu.

In [32]:
import csv
results=pd.DataFrame({"Filename" :filenames,
                      "Predictions":predictions})
                     #orient='index')
    
results.to_csv("prediction_results.csv",index=False)